In [1]:
# from tensorflow.keras.preprocessing import text
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.utils import to_categorical
# from keras.preprocessing import sequence

In [2]:
data = ["Deep learning also known as deep structured learning", 
"is part of a broader family of machine learning methods based", 
"on artificial neural networks with representation learning", 
"Learning can be supervised, semi-supervised or unsupervised",
"Deep-learning architectures such as deep neural networks", 
"deep belief networks, deep reinforcement learning", 
"recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision  speech recognition, natural language processing, machine translation", 
"where they have produced results comparable to and in some cases surpassing human expert performance"
]

In [3]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(data)
word2id = tokenizer.word_index

# build vocabulary of unique words
word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in data]

vocab_size = len(word2id)
embed_size = 100
window_size = 2 # context window size

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 62
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('as', 5), ('of', 6), ('machine', 7), ('supervised', 8), ('and', 9), ('have', 10)]


In [4]:
from keras.utils import pad_sequences
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = to_categorical(label_word, vocab_size)
            yield (x, y)

In [5]:
import numpy as np
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    print(x, y)
    if 0 not in x[0]:
        print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
        i += 1

[[ 0  0  1 12]] [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[ 0  2 12 13]] [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[ 2  1 13  5]] [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Context (X): ['deep', 'learning', 'known', 'as'] -> Target (Y): also
[[ 1 12  5  2]] [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Context (X): ['learning', 'also', 'as', 'deep'] -> Target (Y): known
[[12 13  2 14]] [[0. 0

In [12]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

# build CBOW architecture
cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size))
cbow.add(Lambda(lambda x: tf.reduce_mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# view model summary
print(cbow.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_3 (Lambda)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [14]:
for epoch in range(1, 2):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', 10, '\tLoss:', loss)
    print()

Epoch: 10 	Loss: 337.0563976764679



In [15]:
import pandas as pd
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(61, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,-0.003553,0.012611,-0.101196,0.011609,-0.033450,-0.059699,-0.066600,-0.040408,0.097050,0.089290,...,-0.004583,0.045013,0.037730,0.077474,0.102745,-0.093422,0.055044,-0.130911,0.063891,-0.018388
networks,-0.157056,-0.023990,-0.067739,0.083697,-0.162760,0.041874,-0.114746,-0.122063,0.078052,0.041626,...,0.149940,-0.133434,0.111947,-0.102281,-0.011836,-0.065574,0.205100,-0.024384,-0.042310,-0.054112
neural,-0.144630,0.102455,0.013662,0.109672,0.060890,-0.003770,-0.051117,-0.080582,0.002899,0.167669,...,0.085527,-0.018045,0.096051,0.180708,0.108188,-0.051761,0.056559,-0.003079,-0.065162,0.158613
as,-0.092495,0.011894,0.038813,0.060105,0.001742,0.122750,-0.041913,-0.045183,0.011562,-0.114931,...,0.142746,-0.110932,0.163690,0.090364,-0.087658,0.090705,0.029530,-0.052884,-0.119506,0.077544
of,-0.067710,-0.013423,0.011682,0.088823,-0.027416,-0.019951,-0.058560,-0.068850,0.017068,0.058883,...,-0.025441,0.084034,-0.002163,0.001413,-0.001780,-0.067656,0.066449,0.027973,0.044565,0.017229


In [16]:
from sklearn.metrics.pairwise import euclidean_distances

# compute pairwise distance matrix
distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

# view contextually similar words
similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['deep', 'unsupervised']}

similar_words

(61, 61)


{'deep': ['such', 'reinforcement', 'known', 'architectures', 'also'],
 'unsupervised': ['or', 'semi', 'can', 'produced', 'supervised']}